In [ ]:
from utils import *
import matplotlib.pyplot as plt

img_path = "../img/Outline/"
# Get the folder information
info = load_folder_information(img_path)
# Generate masks
origin = load_image(os.path.join(img_path, info["origin"]))

print(info)
plt.imshow(origin)

In [ ]:
mask_num = len(info["mask_prefix"])
masks = []
for i in range(mask_num):
    mask = np.zeros_like(origin)
    masked_column = origin.shape[1] // (mask_num + 1) * (i + 1)
    mask[:, masked_column:] = 1
    masks.append(mask)

# for line in range(masks[0].shape[0]):
#     for column in range(masks[0].shape[1]):
#         print(masks[0][line][column][0], end=" ")
#     print()
plt.imshow(masks[6]*255)

In [ ]:
# Image files
img_names = os.listdir(img_path)
# MSE
logits = []
for i in range(mask_num):
    mask = masks[i]
    # for line in range(mask.shape[0]):
    #     for column in range(mask.shape[1]):
    #         print(mask[line][column][0], end=" ")
    #     print()
    mask_prefix = info["mask_prefix"][i]
    imgs = [
        load_image(os.path.join(img_path, img_name))
        for img_name in img_names
        if mask_prefix in img_name
    ]
    mse = [image_MSE(origin, img, mask) for img in imgs]
    logits.append(np.mean(mse))
    # for img in imgs:
    #     plt.imshow(img)
    #     plt.show()

print(logits)